In [2]:
import pandas as pd

In [3]:
df_result = pd.read_csv('output/stoks.csv', sep=';')

In [4]:
df_result.head()

,ATIVO,TIPO,DATA,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO,VOLUME
0,BHIA3,acao,16/11/2023,"0,52","11,54","0,58","0,52","0,60","63,10M"
1,BHIA3,acao,14/11/2023,"0,50","4,00","0,52","0,49","0,53","37,90M"
2,BHIA3,acao,13/11/2023,"0,51","-3,85","0,50","0,49","0,53","40,46M"
3,BHIA3,acao,10/11/2023,"0,51","4,00","0,52","0,50","0,54","35,10M"
4,BHIA3,acao,09/11/2023,"0,54","-12,28","0,50","0,50","0,56","70,76M"


In [5]:
df_acao = df_result[df_result['TIPO']=='acao']

In [6]:
df_bdr = df_result[df_result['TIPO']=='bdr']

In [7]:
df_etf = df_result[df_result['TIPO']=='etf']

In [8]:
df_ativo = df_result[df_result['TIPO']=='ativo']

In [9]:
def filter_by_date(df, start_date, end_date):
    """
    Filters a DataFrame based on a date range in the 'dd/mm/yyyy' format.
    
    Parameters:
    - df: DataFrame to be filtered.
    - start_date: Start date in 'dd/mm/yyyy' format.
    - end_date: End date in 'dd/mm/yyyy' format.
    
    Example usage:
    new_df = filter_by_date(df, '02/01/2023', '03/01/2023')
    """
    df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')  # Convert the date column to the datetime format
    return df[(df['DATA'] >= start_date) & (df['DATA'] <= end_date)]

In [10]:
start_date = '02/01/2015'
end_date = '30/05/2023'
new_df = filter_by_date(df_acao, start_date, end_date)

C:\Users\robso\AppData\Local\Temp\ipykernel_19188\578587209.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['DATA'] = pd.to_datetime(df['DATA'], format='%d/%m/%Y')  # Convert the date column to the datetime format


In [11]:
new_df.head()

,ATIVO,TIPO,DATA,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO,VOLUME
117,BHIA3,acao,2023-05-30,"2,40","-2,11","2,32","2,29","2,42","142,52M"
118,BHIA3,acao,2023-05-29,"2,38","0,00","2,37","2,34","2,40","54,63M"
119,BHIA3,acao,2023-05-26,"2,39","0,42","2,37","2,33","2,41","294,12M"
120,BHIA3,acao,2023-05-25,"2,27","7,76","2,36","2,27","2,39","263,87M"
121,BHIA3,acao,2023-05-24,"2,17","0,00","2,19","2,14","2,25","188,31M"


In [12]:
# Certifique-se de que as colunas relevantes sejam do tipo numérico
colunas_numericas = ['ABERTURA', 'FECHAMENTO', 'VARIACAO', 'MINIMO', 'MAXIMO']
df_result[colunas_numericas] = df_result[colunas_numericas].replace(',', '.', regex=True).apply(pd.to_numeric, errors='coerce')

In [13]:
df_result

,ATIVO,TIPO,DATA,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO,VOLUME
0,BHIA3,acao,16/11/2023,0.52,11.54,0.58,0.52,0.60,"63,10M"
1,BHIA3,acao,14/11/2023,0.50,4.00,0.52,0.49,0.53,"37,90M"
2,BHIA3,acao,13/11/2023,0.51,-3.85,0.50,0.49,0.53,"40,46M"
3,BHIA3,acao,10/11/2023,0.51,4.00,0.52,0.50,0.54,"35,10M"
4,BHIA3,acao,09/11/2023,0.54,-12.28,0.50,0.50,0.56,"70,76M"
...,...,...,...,...,...,...,...,...,...
177769,AFLT3,acao,23/11/2021,7.26,0.00,7.25,7.25,7.26,"6,53K"
177770,AFLT3,acao,22/11/2021,7.25,0.00,7.25,7.25,7.25,"10,88K"
177771,AFLT3,acao,19/11/2021,7.25,0.11,7.25,7.25,7.37,"10,90K"
177772,AFLT3,acao,18/11/2021,7.25,0.00,7.25,7.25,7.25,"28,98K"


<h3>Without Map Reduce

In [14]:
# Calcula a média para cada tipo de ação
media_por_tipo = df_result.groupby('TIPO')[colunas_numericas].mean()
media_por_tipo.reset_index(inplace=True)

# Calcula a média para cada ação
media_por_ativo = df_result.groupby('ATIVO')[colunas_numericas].mean()

In [15]:
media_por_tipo

,TIPO,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO
0,acao,16.924249,7.366129,16.978283,16.696253,17.282902
1,ativo,10.329831,-0.014661,10.310677,10.104947,10.537992
2,bdr,62.547210,-0.011962,63.114208,62.576763,63.752420
3,etf,38.230639,-0.071976,38.184457,37.854150,38.613771


<h3> With Map Reduce

In [16]:
# Função para calcular a média de uma lista de valores
from functools import reduce
def calcular_media(valores):
    # Filtra NaNs durante o cálculo da média
    valores_numericos = [v for v in valores if not pd.isna(v)]
    soma = reduce(lambda x, y: x + y, valores_numericos, 0)
    return soma / len(valores_numericos) if len(valores_numericos) > 0 else float('nan')


In [17]:
# Mapeia cada tipo de ação para uma lista de valores relevantes
mapeamento_tipo = df_result.groupby('TIPO')[colunas_numericas].agg(list).to_dict()

# Calcula a média para cada tipo de ação usando map e reduce
medias_por_tipo_mapreduce = {ativo: {coluna: calcular_media(valores) for coluna, valores in dados.items()} for ativo, dados in mapeamento_tipo.items()}


In [18]:
# Mapeia cada tipo de ação para uma lista de valores relevantes
mapeamento_ativo = df_result.groupby('ATIVO')[colunas_numericas].agg(list).to_dict()

# Calcula a média para cada tipo de ação usando map e reduce
medias_por_ativo_mapreduce = {ativo: {coluna: calcular_media(valores) for coluna, valores in dados.items()} for ativo, dados in mapeamento_ativo.items()}


In [22]:
df_map_reduce_ativos = pd.DataFrame(medias_por_ativo_mapreduce)

In [23]:
df_map_reduce_ativos

,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO
A2MC34,9.213807,1.073418,9.070127,8.782611,9.450998
AALR3,18.853840,-0.001514,18.867231,18.593964,19.095120
AALR3F,18.154131,0.048967,18.090025,17.820756,18.475718
AAPL34,41.299920,0.042814,41.276048,40.808144,41.793832
ABTT34,46.958364,-0.065778,46.860346,46.631086,47.116815
...,...,...,...,...,...
XPBR31,109.502475,-0.037066,109.504890,106.929760,112.203992
XRXB34,83.117952,-0.328111,82.292333,81.890444,82.720556
YDRO11,43.128676,-0.103052,42.755040,42.254920,43.421185
Z2NG34,43.898545,0.235862,43.691379,43.587931,43.844138


In [19]:
# from pprint import pprint
# pprint(medias_por_tipo_mapreduce)

df_map_reduce = pd.DataFrame(medias_por_tipo_mapreduce)

In [20]:
# df_map_reduce.rename_axis('TIPO')
df_map_reduce

,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO
acao,16.924249,7.366129,16.978283,16.696253,17.282902
ativo,10.329831,-0.014661,10.310677,10.104947,10.537992
bdr,62.547210,-0.011962,63.114208,62.576763,63.752420
etf,38.230639,-0.071976,38.184457,37.854150,38.613771


In [21]:
media_por_tipo

,TIPO,ABERTURA,FECHAMENTO,VARIACAO,MINIMO,MAXIMO
0,acao,16.924249,7.366129,16.978283,16.696253,17.282902
1,ativo,10.329831,-0.014661,10.310677,10.104947,10.537992
2,bdr,62.547210,-0.011962,63.114208,62.576763,63.752420
3,etf,38.230639,-0.071976,38.184457,37.854150,38.613771
